In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import re
import hex_rating as HR
import pickle
from numpy import asarray

In [2]:
# inital cleaning of german accents, special symbols, and unnecsseary columns
data = pd.read_excel('initial data.xlsx', '50 kx')

data.rename(columns={'IDProbe.L_semittel':'IDProbe.Loesemittel',
                   'IDProbe.Pr_paration':'IDProbe.Preparation',
                  'Vergrößerung':'Vergroesserung'}, inplace=True)

data = data.filter(['IDProbe.Probennummer',
           'IDProbe.Preparation',
           'IDProbe.Loesemittel',
          'IDProbe.Probeninforma1',
           'Bildnummer',
           'Detektor',
           'Vergroesserung',
            'Image_Pixel_S1',
            "Im_BitsPerPixel",
            "Signal",
            'Header_Dump'
          ])

replace_dict = {'ü':'ue','ö':'oe','ä':'ae','μ':'u','µ':'u','\n':'; ',
                'PS88P4VP18':'PS82P4VP18','PS83,6P4VP16,8':'PS83,6P4VP16,4',
                '_x000D_':'','°':''}

for i in data.index:
    for column_name in data.columns:
        strng = data.at[i, column_name]
        if type(strng) is str:
            for origin, new in replace_dict.items():
                    strng = strng.replace(origin,new)
            data.at[i, column_name] = strng
            
            

In [3]:
# splits column 'IDProbe.Probeninforma1' and its information to many different independent columns
def split_column(df, column_label, row_index):
    
    row_val = df.iloc[row_index][column_label]
    
    if not pd.isnull(row_val): 
        new_vals = row_val.split(';') # splits every row

        nv_dict = {} #the keys represent new column names, the values represent new entries, all originated from 'IDProbe.Probeninforma1'
        for elem in new_vals:
            if ':' in elem:
                key, value = elem.split(':',1)
                nv_dict[key.strip()] = value.strip()
            elif "Recycled" in elem:
                nv_dict['Is Recycled'] = elem
                

        for key,value in nv_dict.items():
            df.loc[row_index,key] = value
                

In [4]:
 #execution of the function above
for i in data.index:
    split_column(data, 'IDProbe.Probeninforma1', i)
    
# labels that are interesting to add later, but have relatively few filled rows:
#electric field, casting, bladegap, Luftfeuchtigkeit, Temperatur, 
#even later, temperaturbehandlung after crosslinking should be checked

In [5]:
# checks if given data of image is compatible to requirements
    
properties = {
    "Image_Pixel_S1":['2.233 nm'],
    "Vergroesserung":[50,50.0],
    "Im_BitsPerPixel":[8],
    "Detektor":["InLens"],
    "Signal":[1.0],
        }

# checks every label and its value for each row in the data.
# if value is not valid, the row is dropped out
for index, row in data.iterrows():
    for label_name, values in properties.items():
        if row[label_name] not in values:
            #print(row["Bildnummer"]+", "+str(index)+":"+label_name+":"+str(row[label_name])+" is not valid")
            data.drop(index, inplace=True)
            break
            
data.drop(865, inplace=True) # deletes duplicated image in the database

In [6]:
# some rows have 'Abdampfzeit N2' and 'Abdampfzeit in Air' instead of 'Abdampfzeit Air'
#(we treat them as they're the same) therefore we transfer them to 'Abdampfzeit Air' and drop labels 
# of 'Abdampfzeit N2' and 'Abdampfzeit in Air'

for i in data.index:
    if not pd.isnull(data.at[i, 'Abdampfzeit N2']):
        data.at[i, 'Abdampfzeit Air'] = data.at[i, 'Abdampfzeit N2']
        
for i in data.index:
    if not pd.isnull(data.at[i, 'Abdampfzeit in Air']):
        data.at[i, 'Abdampfzeit Air'] = data.at[i, 'Abdampfzeit in Air']
        
data.drop(columns = ['Abdampfzeit N2', 'Abdampfzeit in Air'], inplace = True)

In [7]:
# some rows have values in 'Abdampfzeit E-Feld' and in 'Abdampfzeit Air' (we treat them as they're the same)
# therefore the first one is added to the second one, respectively.
 
for i in data.index:
    
    if not pd.isnull(data.at[i, 'Abdampfzeit E-Feld']) and data.at[i, 'Abdampfzeit E-Feld'] != '0s':
        seconds_air = int(data.at[i, 'Abdampfzeit Air'].replace('s',''))
        seconds_efeld = int(data.at[i, 'Abdampfzeit E-Feld'].replace('s',''))
        seconds_air = seconds_air+seconds_efeld
        data.at[i, 'Abdampfzeit Air'] = seconds_air

In [8]:
# getting rid of non relevant columns
data = data.filter([
                    'Bildnummer',
                    'Polymerkonzentration',
                    'Loesemittel',
                    'Abdampfzeit Air',
                    'Zusammensetzung',
                    ])

In [9]:
def recognize_data(re_list,label_name):
    # given regex list and column name, gets data from 'bildnummer' column and sets it in designated column.
    # the alogrithm is based on matching every entry to regular expressions
    # as theres a match, the entry is saved in a designated column
    
    data[label_name] = np.nan # creates an empty new column
    
    for i in data.index:
        strng = data.at[i, 'Bildnummer']
        for exp in re_list:
            res = re.findall(exp, strng, re.IGNORECASE) # res stores the extracted data from 'bildnummer' column
            
            if res:
                #print(label_name, res, [strng.replace(elem,'@@@') for elem in res])# usefull for residue check
                data.loc[i, label_name] = res[0]
                break
                
        # this part is dedicated to fill empty entries of 'ps_p4vp_ratio' and 'molec_weight'
        # which have mathching values in 'Zusammensetzung' column
        if label_name in ['ps_p4vp_ratio', 'molec_weight']:
            if not pd.isnull(data.at[i, 'Zusammensetzung']) and pd.isnull(data.at[i, label_name]):
                strng = data.at[i, 'Zusammensetzung']
                for exp in re_list:
                    res = re.findall(exp, strng, re.IGNORECASE)
                    if res:
                        data.at[i, label_name] = res[0]

In [10]:
# the lists are regular expressions input patterns to the function above.
# besides them, theres a line where the function is executed with them
solution = [ r'THF_DMF_DOX_\d+_\d+_\d+', r'\d+DMF_\d+THF_\d+DOX',r'\d+DMF_\d+THF', r'\d+DMF-\d+THF', r'\d+THF_\d+DMF',
            r'THF_DMF_\d+_\d+', r'DMF_THF_\d+_\d+', r'DMF-THF_\d+-\d+', r'DMF-THF_\d+-\d+', r'DMF\d+-THF\d+',
            r'\d+DOX_\d+THF', r'DMF\d+THF\d+', r'THF\d+DMF\d+', r'\d+D_\d+T;', '100DOX','DOX100', 'D_T;', 'D_T',
            '6-4','4-6','5-5', 'THF_DMF', 'DMF_THF', 'DMF;']

recognize_data(solution,'solution')

evap_air = [r'_\d+sAir_', r'_\d+sAir_',
           r'_\d+sec_', r'_\d+sec_',r'_\d+sec-', r'_\d+sec-', 
            r'_\d+s_', r'_\d+s_', r'-\d+s_', r'-\d+s_', r'-\d+s-', r'-\d+s-', r'S\d+s', r'S\d+s', r'G\d+s',
            r'G\d+s', r'S\d+.\d+s']

recognize_data(evap_air,'evap_air')

poly_concen = [r'_\d+.\d+ wt%_', r'\d+wt%', r'\d+.\d+wt%',
             r'\d+ wt%', 'K(\d+)-', r'_\d+%_', r'_\d+%_', r'_\d+.\d+%_', r'_\d+%', r'-\d+%-']
                

recognize_data(poly_concen,'poly_concen')

ps_p4vp_ratio = [r'PS\d+P4VP\d+', r'PS\d+.\d+P4VP\d+.\d+', r'PS\d+-P4VP\d+', r'S\d+VP\d+', r'S\d+VP\d+',
                r'PVBCB\d+P4VP\d+', r'PS\d+.\d+_P4VP\d+.\d+', r'PS\d+_P4VP\d+',r'PS-4VP\d+',
                 r'PVBCB\d+-P4VP\d+', r'\d+P4VP\d+', 'S\d+_4VP\d+', r'S\d+VP', r'PS\d+-b-P4VP\d+',
                 r'PSP4VP\d+', r'SVP\d+', r'\d+PS, \d+ P4VP']

recognize_data(ps_p4vp_ratio,'ps_p4vp_ratio')

molec_weight = ['98_0kV', r'\d+k', r'-\d+ ', r'-\d+-', r'\d+_0kV', r'-\d+_recycle', '-154_', '_197_']

recognize_data(molec_weight,'molec_weight')

In [11]:
# every entry is strapped of non relevant characters in Polymerkonzentration and poly_concen columns
def poly_concen_remover(label_name):
    remove_list = ['%','-','_','wt', 'c', 'Wt']
    for i in data.index:
        if not pd.isnull(data.at[i, label_name]) and type(data.at[i, label_name]) is str:
            for strng in remove_list:
                data.at[i, label_name] = data.at[i, label_name].replace(strng,'')
            data.at[i, label_name] = data.at[i, label_name].replace(',','.')
            data.at[i, label_name] = float(data.at[i, label_name])

poly_concen_remover('poly_concen')
poly_concen_remover('Polymerkonzentration')

In [12]:
# every entry is strapped of non relevant characters in Abdampfzeit Air and evap_air columns
def evap_air_remover(label_name):
    remove_list = ['S','s', 'G', 'sAir', 'Air','ec', '_', '-', '>']
    for i in data.index:
        if not pd.isnull(data.at[i, label_name]) and type(data.at[i, label_name]) is str:
            for strng in remove_list:
                data.at[i, label_name] = str(data.at[i, label_name]).replace(strng,'')
            data.at[i, label_name] = data.at[i, label_name].replace(',','.')
            data.at[i, label_name] = float(data.at[i, label_name])
            
evap_air_remover('evap_air')
evap_air_remover('Abdampfzeit Air')

In [13]:
# every entry is strapped of non relevant characters in 'molec_weight' columnz
remove_list = ['recycle', '_0kV', 'k', 'K', '_', '-','^']
for i in data.index:
    if not pd.isnull(data.at[i, 'molec_weight'])and type(data.at[i, 'molec_weight']) is str:
        for strng in remove_list:
            data.at[i, 'molec_weight'] = data.at[i, 'molec_weight'].replace(strng,'')
        data.at[i, 'molec_weight'] = float(data.at[i, 'molec_weight'])

In [14]:
# fills empty entries, which their value was found after spliting 'bildnummer' column above

# 'x_fill' are dataframes for eye-checking if extracted values are okay

solution_fill = pd.DataFrame(columns = ['Bildnummer' ,'Polymerkonzentration' ,'Loesemittel' ,'Abdampfzeit Air' ,
                                     'Zusammensetzung' ,'solution' ,'evap_air' ,'poly_concen' ,'ps_p4vp_ratio' ,
                                     'molec_weight'])
evap_air_fill = pd.DataFrame(columns = ['Bildnummer' ,'Polymerkonzentration' ,'Loesemittel' ,'Abdampfzeit Air' ,
                                     'Zusammensetzung' ,'solution' ,'evap_air' ,'poly_concen' ,'ps_p4vp_ratio' ,
                                     'molec_weight'])
poly_concen_fill = pd.DataFrame(columns = ['Bildnummer' ,'Polymerkonzentration' ,'Loesemittel' ,'Abdampfzeit Air' ,
                                     'Zusammensetzung' ,'solution' ,'evap_air' ,'poly_concen' ,'ps_p4vp_ratio' ,
                                     'molec_weight'])


for i in data.index:

    if pd.isnull(data.at[i, 'Loesemittel']) and (not pd.isnull(data.at[i, 'solution'])):
        row_dict = data.iloc[i].to_dict()
        solution_fill = solution_fill.append(row_dict, ignore_index = True)
        data.at[i, 'Loesemittel'] = data.at[i, 'solution']
        
    if pd.isnull(data.at[i, 'Abdampfzeit Air']) and (not pd.isnull(data.at[i, 'evap_air'])):
        row_dict = data.iloc[i].to_dict()
        evap_air_fill = evap_air_fill.append(row_dict, ignore_index = True)
        data.at[i, 'Abdampfzeit Air'] = data.at[i, 'evap_air']
        
    if pd.isnull(data.at[i, 'Polymerkonzentration']) and (not pd.isnull(data.at[i, 'poly_concen'])):
        row_dict = data.iloc[i].to_dict()
        poly_concen_fill = poly_concen_fill.append(row_dict, ignore_index = True)
        data.at[i, 'Polymerkonzentration'] = data.at[i, 'poly_concen']
        
#solution_fill.filter(['Bildnummer','solution']).to_csv('solution_fill.csv')
#evap_air_fill.filter(['Bildnummer','evap_air']).to_csv('evap_air_fill.csv')
#poly_concen_fill.filter(['Bildnummer','poly_concen']).to_csv('poly_concen_fill.csv')

In [15]:
# this code is devoted for eye-checking of human mistakes, mostly differences between information in 'Bildnummer'
# and extracted data from 'Bildnummer' column

evap_air_comparison = pd.DataFrame(columns = ['Bildnummer' ,'Polymerkonzentration' ,'Loesemittel' ,'Abdampfzeit Air' ,
                                     'Zusammensetzung' ,'solution' ,'evap_air' ,'poly_concen' ,'ps_p4vp_ratio' ,
                                     'molec_weight'])
poly_concen_comparison = pd.DataFrame(columns = ['Bildnummer' ,'Polymerkonzentration' ,'Loesemittel' ,'Abdampfzeit Air' ,
                                     'Zusammensetzung' ,'solution' ,'evap_air' ,'poly_concen' ,'ps_p4vp_ratio' ,
                                     'molec_weight'])

for i in data.index:     
    if (not pd.isnull(data.at[i, 'Abdampfzeit Air'])) and (not pd.isnull(data.at[i, 'evap_air'])):
        if data.at[i, 'Abdampfzeit Air'] != data.at[i, 'evap_air']:
            row_dict = data.iloc[i].to_dict()
            evap_air_comparison = evap_air_comparison.append(row_dict, ignore_index = True)

    if (not pd.isnull(data.at[i, 'Polymerkonzentration'])) and (not pd.isnull(data.at[i, 'poly_concen'])):
        if data.at[i, 'Polymerkonzentration'] != data.at[i, 'poly_concen']:
            row_dict = data.iloc[i].to_dict()
            poly_concen_comparison = evap_air_comparison.append(row_dict, ignore_index = True)


#evap_air_comparison.filter(['Bildnummer','Abdampfzeit Air','evap_air']).to_csv('evap_air_comparison.csv')
#poly_concen_comparison.filter(['Bildnummer','Polymerkonzentration','poly_concen']).to_csv('poly_concen_comparison.csv')

In [16]:
# completion of some missing data, which was collected manually
complement_data = pd.read_csv('manually_added_missing.csv',index_col=0)

for i, row in complement_data.iterrows():
    
    file_name = row[0]
    index = data.loc[data['Bildnummer']==file_name].index[0]
    
    data.at[index, 'molec_weight'] = complement_data.at[i, 'molec_weight']
    data.at[index, 'Abdampfzeit Air'] = complement_data.at[i, 'Abdampfzeit Air']
    data.at[index, 'Loesemittel'] = complement_data.at[i, 'Loesemittel']
    data.at[index, 'Polymerkonzentration'] = complement_data.at[i, 'Polymerkonzentration']

# removal of rows with missing data which cant be filled
missing_data = pd.read_csv('missing data.csv', index_col=0)
for row in missing_data.iterrows():
    file_name = row[1][0]
    data = data[data['Bildnummer'] != file_name]

In [276]:
# every entry in 'ps_p4vp_ratio' is matched to a single form of entry: XX in 'PS4VP_[%]' column(XX is numbers)

#patterns indices:
                 #0-10
remove_list = [r'(\d+)PS. (\d+) P4VP',r'PS(\d+)P4VP(\d+)', r'PS(\d+.\d+)P4VP(\d+.\d+)', r'PS(\d+)-P4VP(\d+)', r'S(\d+)VP(\d+)',
                r'PVBCB(\d+)P4VP(\d+)', r'PS(\d+.\d+)_P4VP(\d+.\d+)', r'PS(\d+)_P4VP(\d+)',
                 r'PVBCB(\d+)-P4VP(\d+)', r'(\d+)P4VP(\d+)', 'S(\d+)_4VP(\d+)',
                 #11-15
               r'PS(\d+)-b-P4VP(\d+)',r'PS-4VP(\d+)',r'PSP4VP(\d+)', r'SVP(\d+)', r'(\d+)PS, (\d+) P4VP',
                 #16
               r'S(\d+)VP']

data['P4VP_[%]'] = np.nan

for row in data.iterrows():
    j = row[0] # just index of a row
    if not pd.isnull(data.at[j, 'ps_p4vp_ratio']):
        strng = data.at[j, 'ps_p4vp_ratio'].replace(',','.') # some filenames have ',' instead of '.' for decimal numbers
        for i in range(len(remove_list)):
            result = re.findall(remove_list[i], strng, re.IGNORECASE) # stores the extracted numbers from 'ps_p4vp_ratio' entries
            if not result:
                continue
            else:
                percentages = result[0]
                
                # i represents the index  of the right regular experssion pattern of the entry
                # in the order of the patterns in 'remove_list'.
                if type(percentages) is tuple: # if there two numbers in the entry, patterns indices 0-10
                    per_ps_pvbcb, per_p4vp = percentages[0], percentages[1]
                    data.at[j, 'P4VP_[%]'] = float(per_p4vp)
                    break

                else: # if there is a single number in the entry
                    if i in [11,12, 13, 14, 15]:
                        per_p4vp = percentages
                    if i == 16:
                        per_ps = percentages
                        per_p4vp = str(100-int(per_ps))
                    data.at[j, 'P4VP_[%]'] = float(per_p4vp)
                    break

In [277]:
# every entry in 'Loesemittel' is matched to a single form of entry:
#YY in 'DMF_[%]' column and YY in 'THF_[%]' column (YY are numbers)


# patterns index:
                 #0-9
remove_list = [r'DMF(\d+)-THF(\d+.\d+)-Dioxan(\d+.\d+)', r'DMF(\d+)-THF(\d+)-Dioxan(\d+)', r'(\d+)DMF-(\d+)THF-(\d+)Dioxan', r'DMF/THF/Dioxan (\d+)/(\d+)/(\d+)',
               r'(\d+)THF-(\d+)DMF-(\d+)Dioxan', r'(\d+)THF/(\d+)DMF/(\d+)Dioxan', r'THF/DMF/Dioxan : (\d+)/(\d+)/(\d+)',
                r'(\d+) THF/ (\d+) DMF/ (\d+) Dioxan', r'THF_DMF_DOX_(\d+)_(\d+)_(\d+)',
               r'DMF(\d+)-Dioxan(\d+)-THF(\d+)',
                #10-23
               r'(\d+)DMF_(\d+)THF', r'(\d+)DMF-(\d+)THF', r'DMF(\d+)THF(\d+)',r'(\d+)DMF/(\d+)THF',
               r'DMF(\d+)/THF(\d+)', r'DMF(\d+)-THF(\d+)', r'(\d+)DMF-THF(\d+)', r'DMF/THF (\d+)/(\d+)',
               r'THF(\d+)-DMF(\d+)', r'(\d+)THF-(\d+)DMF', r'(\d+) THF/ (\d+) DMF',
                r'THF/DMF : (\d+)/(\d+)', r'THF/DMF, (\d+):(\d+)', r'(\d+)THF/(\d+)DMF',
                #24-25
                'THF-DMF', 'THF/DMF',
                #26
               r'DOX(\d+)/THF(\d+)',
                #27
               'DMF',
                #28-29
               '100DOX','Dioxan']

data['DMF_[%]'], data['THF_[%]'] = 0, 0
# outer loop - extracts  and fixes numbers from 'Loesemittel' entries
# inner loop - saves the numbers in a designated column
for row in data.iterrows():
    j = row[0]
    if not pd.isnull(data.at[j, 'Loesemittel']):
        
        strng = data.at[j, 'Loesemittel'].replace(',','.')

        percents = re.findall(r'\d+', strng, re.IGNORECASE)

        if len(percents) == 2: # if there are two numbers in the entry
            if len(percents[0])==1 and len(percents[1])==1:
                if percents[0] == '1' and percents[1] == '1':
                    fix_per1, fix_per2 = '50', '50'
                elif percents[0] == '2' and percents[1] == '3':
                    fix_per1, fix_per2 = '40', '60'
                elif percents[0] == '3' and percents[1] == '2':
                    fix_per1, fix_per2 = '60', '40'
                else:
                    fix_per1, fix_per2 = percents[0]+'0', percents[1]+'0' # some entries are just to be filled with another zero
                strng = strng.replace(percents[0],fix_per1)
                if percents[0] != '1' and percents[0] != '5':
                    strng = strng.replace(percents[1],fix_per2)

        if len(percents) == 3: # if there are three numbers in the entry
            if len(percents[0]) == 1 and len(percents[1]) == 1 and len(percents[2]) == 1:
                fix_per1, fix_per2, fix_per3 = percents[0]+'0', percents[1]+'0', percents[2]+'0'
                strng = strng.replace(percents[0],fix_per1)
                strng = strng.replace(percents[1],fix_per2)
                
        for i in range(len(remove_list)):
            result = re.findall(remove_list[i], strng, re.IGNORECASE)
            if result:
                # i represents the index  of the right regular experssion pattern of the entry
                # in the order of the patterns in 'remove_list'.
                if 0<=i<=9:
                    if 0<=i<=3:
                        dmf_per, thf_per, dox_per = result[0][0], result[0][1], result[0][2]
                    if 4<=i<=8:
                        thf_per, dmf_per, dox_per = result[0][0], result[0][1], result[0][2]
                    if i==9:
                        dmf_per, dox_per, thf_per = result[0][0], result[0][1], result[0][2]
                    data.at[j, 'DMF_[%]'], data.at[j, 'THF_[%]'] = float(dmf_per), float(thf_per)
                  
                
                if 10<=i<=23:
                    if 10<=i<=17:
                        dmf_per, thf_per = result[0][0], result[0][1]
                    if 18<=i<=23:
                        thf_per, dmf_per = result[0][0], result[0][1]
                    data.at[j, 'DMF_[%]'], data.at[j, 'THF_[%]'] = float(dmf_per), float(thf_per)
                    
                if 24<=i<=25:
                    thf_per, dmf_per = str(50), str(50)
                    data.at[j, 'DMF_[%]'], data.at[j, 'THF_[%]'] = float(dmf_per), float(thf_per)
                
                if i==26:
                    dox_per, thf_per = result[0][0], result[0][1]
                    data.at[j, 'THF_[%]'] = float(thf_per)
                    
                if i == 27:
                    dmf_per = str(100)
                    data.at[j, 'DMF_[%]'] = float(dmf_per)
                    
                if 28<=i<=30:
                    pass # no action beacuase 0 thf and 0 dmf menas 100 dox
                    
                break


In [278]:
# getting rid of temporary used columns
data.drop(columns = ['Zusammensetzung', 'solution', 'evap_air', 'poly_concen',
                    'Loesemittel', 'ps_p4vp_ratio'], inplace = True)
# converting string written numbers to floats
for label in ['Polymerkonzentration', 'Abdampfzeit Air', 'molec_weight']:
    data[label] = pd.to_numeric(data[label], downcast="float")
# renaming column names
data.rename(columns={'Polymerkonzentration': 'Polymer_Concentration_[%]','Abdampfzeit Air': 'Evaporation_Time_[s]',
                     'molec_weight': 'Molecular_Weight_[k]', 'Bildnummer': 'File_name'}, inplace=True)
# and finally saving the processed data
data.to_csv('processed_data.csv')